# **Sentiment Index Generation: Llama**

# **All Installations**

In [43]:
%pip install transformers datasets peft accelerate evaluate torch --quiet

In [44]:
%pip install python-dotenv

In [45]:
import os
import platform

def data_download(file_to_download, gdrive_code, OS, uncompress = True):
  if not os.path.exists(file_to_download):
    os.system('gdown --id "'+gdrive_code+'" --output '+file_to_download)
    if OS == "Linux" and uncompress:
        os.system('unzip -o -n "./'+file_to_download+'" -d "./"')
    return True
  else:
    return None

OS = platform.system()

out = data_download("./Sentiment_Dataset.zip", "1OEQpkzN6HMcZc3yPg8G0ahvh6lOCMlIR", OS)

# **All Imports**

In [46]:
#General
import torch
import random
import calendar
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import plotly.express as px

# LLMs
import evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# **All Globals**

In [90]:
seed = 42
dataset_limit = 1000
sentiment_index_limit = 100
n_tweets_per_day = 20
n_epochs = 5
moving_average_window = 7
llm_model_parameters_number = '1B'                # number of billion parameters of the LLM mode, 1B is the smaller llama3.2 1 billion of parameters
model_name = f"meta-llama/Llama-3.2-{llm_model_parameters_number}"
dataset_path = "./Sentiment_Dataset/"

# **Initializations**

In [48]:
os.environ["WANDB_MODE"] = "disabled"

!mv "{dataset_path}.env" ./
random.seed(seed)
np.random.seed(seed)

load_dotenv()

mv: cannot stat './Sentiment_Dataset/.env': No such file or directory


True

# **Functions Definition**

In [49]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

def df_predictions(df, tokenized_df, trainer, label_mapping):
    predictions = trainer.predict(tokenized_df)
    predicted_labels = predictions.predictions.argmax(-1)
    label_mapping = np.load("labels_mapping.npy", allow_pickle=True).item()
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    predicted_sentiment = [reverse_label_mapping[label] for label in predicted_labels]
    df['predicted_sentiment'] = predicted_sentiment
    return df

def month_sort_key(month_dir):
    month_name = os.path.basename(month_dir).split('_')[0]
    try:
        return list(calendar.month_name).index(month_name.capitalize())
    except ValueError:
        return 0

# **Dataset Reading**

In [50]:
irony_df = pd.read_csv("./Sentiment_Dataset/irony_dataset.txt", header=None).rename(columns={0: 'text'})["text"].to_frame()

In [51]:
df = pd.read_csv("./Sentiment_Dataset/training_set.csv", sep = ";")
df = df[["text", "pol"]].dropna()         # select the columns interesting for us and drop rows with NaN

In [52]:
if dataset_limit is not None:
    df = df.sample(dataset_limit).reset_index(drop=True)

In [53]:
print("Numnber of data-points:", len(df))

Numnber of data-points: 100


In [54]:
df.head()

,text,pol
0,Arbasina/Giavazzi sul Corriere a proposito del...,0.0
1,il trading è un amp azienda ecco come risparmi...,1.0
2,"RT TAG mi hanno unfollowato in massa, mortacci...",2.0
3,modalità prezzi e biglietti per la gara al man...,0.0
4,donnastef shop t shirt gonna borsa spedizione ...,0.0


1: 'Positive', 2: 'Negative', 0: 'Neutral'

In [55]:
print("Number of classes", df["pol"].nunique())
print("Types of classes", df["pol"].unique())

Number of classes 3
Types of classes [0. 1. 2.]


# **Dataset Recoding**

In [56]:
df['pol'] = df['pol'].map({1: 'Positive', 2: 'Negative', 0: 'Neutral'})
df = df.rename(columns={'pol': 'label'})

In [57]:
df.head()

,text,label
0,Arbasina/Giavazzi sul Corriere a proposito del...,Neutral
1,il trading è un amp azienda ecco come risparmi...,Positive
2,"RT TAG mi hanno unfollowato in massa, mortacci...",Negative
3,modalità prezzi e biglietti per la gara al man...,Neutral
4,donnastef shop t shirt gonna borsa spedizione ...,Neutral


# **Label Conversion**

In [58]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
np.save("labels_mapping.npy", label_mapping)
print("Label mapping:", label_mapping)

Label mapping: {'Negative': np.int64(0), 'Neutral': np.int64(1), 'Positive': np.int64(2)}


# **Train-Test Split**

In [59]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# **Dataset Conversion for Transformers**

In [60]:
irony_dataset = Dataset.from_pandas(irony_df)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
unique_labels = train_dataset.unique('label')
num_labels = len(unique_labels)
print("Number of Labels withing the Training Set:", len(unique_labels))

Number of Labels withing the Training Set: 3


In [61]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 80
})

In [62]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 20
})

# **Tokenization and Padding**

In [63]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [64]:
tokenized_irony = irony_dataset.map(tokenize_function, batched=True)
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

## **Load the LLM Model**

In [65]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Definition of QLora**

In [66]:
# === 7. Applica LoRA (fine-tuning efficiente) ===
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, peft_config)

 # **Metrics Definition**

In [67]:
accuracy = evaluate.load("accuracy")

In [68]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

# **Training Parameters Definition**

In [69]:
training_args = TrainingArguments(
    output_dir="./llama3_finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=n_epochs,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    report_to=[]  # Disactivate the W&B
)

# **Training Model Definition**

In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-3389762461.py:1: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [71]:
#irony_df = df_predictions(irony_df, tokenized_irony, trainer, label_mapping)

In [72]:
#irony_df

In [73]:
'''
negative_count = irony_df[irony_df['predicted_sentiment'] == 'Negative'].shape[0]
total_count = irony_df.shape[0]
percentage_negative = (negative_count / total_count) * 100

print(f"Percentage of negative sentiments detected in irony_df after training: {percentage_negative:.2f}%")
'''

'\nnegative_count = irony_df[irony_df[\'predicted_sentiment\'] == \'Negative\'].shape[0]\ntotal_count = irony_df.shape[0]\npercentage_negative = (negative_count / total_count) * 100\n\nprint(f"Percentage of negative sentiments detected in irony_df after training: {percentage_negative:.2f}%")\n'

# **Model Training**

In [74]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.778906,0.400000
2,2.121500,1.667383,0.400000
3,1.804700,1.638965,0.450000


TrainOutput(global_step=120, training_loss=1.9580647786458334, metrics={'train_runtime': 128.0779, 'train_samples_per_second': 1.874, 'train_steps_per_second': 0.937, 'total_flos': 359059182059520.0, 'train_loss': 1.9580647786458334, 'epoch': 3.0})

# **Model Evaluation**

In [75]:
results = trainer.evaluate()
print(f"\n✅ Accuracy sul test set: {results['eval_accuracy']:.4f}")


✅ Accuracy sul test set: 0.4500


# **Model Testing on Irony Dataset**

In [76]:
irony_df = df_predictions(irony_df, tokenized_irony, trainer, label_mapping)

In [77]:
irony_df

,text,predicted_sentiment
0,"Oh, che meraviglia, ancora una volta il treno ...",Neutral
1,La mia giornata è stata così fantastica che st...,Neutral
2,Sono così grato al mio capo per avermi assegna...,Neutral
3,"Wow, sono così felice che il mio vicino abbia ...",Neutral
4,Adoro quando il mio computer decide di bloccar...,Neutral
...,...,...
66,Finalmente ho trovato il tempo per riposarmi e...,Neutral
67,La mia squadra del cuore ha appena pareggiato ...,Neutral
68,"Oh, fantastico, ho dimenticato il mio portafog...",Neutral
69,Sono così fortunato ad avere una sveglia così ...,Neutral


In [78]:
negative_count = irony_df[irony_df['predicted_sentiment'] == 'Negative'].shape[0]
total_count = irony_df.shape[0]
percentage_negative = (negative_count / total_count) * 100

print(f"Percentage of negative sentiments detected in irony_df after training: {percentage_negative:.2f}%")

Percentage of negative sentiments detected in irony_df after training: 2.82%


# **Sentiment Index Creation**

## **Tweets Production Set Reading**

In [91]:
all_dataframes = []
directory_path = "./Sentiment_Dataset/2020_index_tweets_set/"
month_directories = [os.path.join(directory_path, d) for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
month_directories.sort(key=month_sort_key) # Sort the directories chronologically

for month_dir in month_directories:
    print("Month File: ", month_dir)
    csv_files = [f for f in os.listdir(month_dir) if f.endswith('.csv')]
    # Sort csv files by date in filename
    csv_files.sort(key=lambda x: x.split('.')[0])
    for file_name in csv_files:
        print("Day File: ", file_name)
        file_path = os.path.join(month_dir, file_name)
        try:
            df = pd.read_csv(file_path, encoding='utf-8', nrows = n_tweets_per_day, sep = ';')
        except UnicodeDecodeError:
            df = pd.read_csv(file_path, encoding='latin1', nrows = n_tweets_per_day, sep = ';')
        all_dataframes.append((file_name.split('.')[0], df["testo"]))

Month File:  ./Sentiment_Dataset/2020_index_tweets_set/jan_tweets_social_mood
Day File:  2020-01-01.csv
Day File:  2020-01-02.csv
Day File:  2020-01-03.csv
Day File:  2020-01-04.csv
Day File:  2020-01-05.csv
Day File:  2020-01-06.csv
Day File:  2020-01-07.csv
Day File:  2020-01-08.csv
Day File:  2020-01-09.csv
Day File:  2020-01-10.csv
Day File:  2020-01-11.csv
Day File:  2020-01-12.csv
Day File:  2020-01-13.csv
Day File:  2020-01-14.csv
Day File:  2020-01-15.csv
Day File:  2020-01-16.csv
Day File:  2020-01-17.csv
Day File:  2020-01-18.csv
Day File:  2020-01-19.csv
Day File:  2020-01-20.csv
Day File:  2020-01-21.csv
Day File:  2020-01-22.csv
Day File:  2020-01-23.csv
Day File:  2020-01-24.csv
Day File:  2020-01-25.csv
Day File:  2020-01-26.csv
Day File:  2020-01-27.csv
Day File:  2020-01-28.csv
Day File:  2020-01-29.csv
Day File:  2020-01-30.csv
Day File:  2020-01-31.csv
Month File:  ./Sentiment_Dataset/2020_index_tweets_set/february_tweets_social_mood
Day File:  2020-02-01.csv
Day Fil

In [92]:
all_dataframes[0]

('2020-01-01',
 0     per la cronaca canova fu uno dei primi monumen...
 1     l opera d arte del giorno antonio canova amore...
 2     rt borghi claudio iniziare l anno con uno che ...
 3     tutta salute approda in seconda serata per ini...
 4     un anno di lavoro se permetti lo auguro io a t...
 5     senatore salvini sicuro che con quota restitui...
 6     aleguerani davidevignes luzziandrea certamente...
 7     sto andando al pranzo di famiglia in mezzo ai ...
 8     la raggi chiude l anno con bilancio in positiv...
 9     ullallà il simpatico senatore castiello che ha...
 10    botti e spari il bilancio della notte di capod...
 11    gennaio come nuova moneta in italia la vollero...
 12    roberto e roberto repost officialrobertobolle ...
 13    ho avuto come la sensazione che ci stessero fa...
 14    veroderomanis è inutile ragionare con chi ha i...
 15    conte ci ha preso gusto il m s no la paladina ...
 16    giomasmic si è visto con il qe migliaia di mil...
 17    gennaio c

In [93]:
daily_sentiment_counts = []

i = 0
for date, series in all_dataframes:
    print(f"Computing day {date}")
    df = series.to_frame(name='text')
    dataset = Dataset.from_pandas(df)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    predictions = trainer.predict(tokenized_dataset)
    predicted_labels = predictions.predictions.argmax(-1)
    label_mapping = np.load("labels_mapping.npy", allow_pickle=True).item()
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    predicted_sentiment = [reverse_label_mapping[label] for label in predicted_labels]
    positive_count = predicted_sentiment.count('Positive')
    negative_count = predicted_sentiment.count('Negative')
    neutral_count = predicted_sentiment.count('Neutral')
    daily_sentiment_counts.append({'date': date, 'positive': positive_count, 'negative': negative_count, "sentiment_index": (positive_count-negative_count)/(positive_count+neutral_count+negative_count)})
    i += 1
    if i>=sentiment_index_limit and sentiment_index_limit is not None:
      break

sentiment_counts_df = pd.DataFrame(daily_sentiment_counts)
display(sentiment_counts_df)

Computing day 2020-01-01


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Computing day 2020-01-02


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Computing day 2020-01-03


...

Computing day 2020-04-08


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Computing day 2020-04-09


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

,date,positive,negative,sentiment_index
0,2020-01-01,0,3,-0.15
1,2020-01-02,0,3,-0.15
2,2020-01-03,0,2,-0.10
3,2020-01-04,0,3,-0.15
4,2020-01-05,0,0,0.00
...,...,...,...,...
95,2020-04-05,0,2,-0.10
96,2020-04-06,0,1,-0.05
97,2020-04-07,0,3,-0.15
98,2020-04-08,0,3,-0.15


In [94]:
sentiment_counts_df.to_excel("./sentiment_index.xlsx")

# **Plot of the Setiment Index**

In [97]:
sentiment_counts_df['rolling_avg'] = sentiment_counts_df['sentiment_index'].rolling(window=moving_average_window).mean()

fig = px.line(sentiment_counts_df, x='date', y='sentiment_index', title='Daily Sentiment Index')
fig.add_scatter(x=sentiment_counts_df['date'], y=sentiment_counts_df['rolling_avg'], mode='lines', name='7-Day Rolling Average', line=dict(color='red'))
fig.update_layout(xaxis_title='Date', yaxis_title='Sentiment Index')
fig.update_xaxes(tickformat="%Y-%m-%d", tickangle=45) # Format date and rotate labels
fig.write_html("./sentiment_index_plot.html")
#fig.write_image("./sentiment_index_plot.jpeg")
fig.show()